## 6-7. 프로젝트: 멋진 작사가 만들기

### 실습
*  데이터 다듬기
*  인공지능 학습시키기
*  잘 만들어졌는지 평가하기

### 1. 데이터 읽어오기

In [48]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import glob                # glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요.
import os

txt_file_path = os.getenv('HOME')+'/aiffel/e6.lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['\ufeffEighteen years eighteen years', 'She got one of your kids got you for eighteen years', 'I know somebody paying child support for one of his kids']


In [49]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

﻿Eighteen years eighteen years
She got one of your kids got you for eighteen years
I know somebody paying child support for one of his kids
His baby mama car and crib is bigger than his
You will see him on TV any given Sunday
Win the Super Bowl and drive off in a Hyundai
She was supposed to buy your shorty Tyco with your money
She went to the doctor got lipo with your money
She walking around looking like Michael with your money
Shouldve got that insured Geico for your money


### 2.데이터 정제

In [50]:
#데이터셋 정제
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.


<start> this is sample sentence . <end>


In [51]:
corpus = []

for sentence in raw_corpus:
    corpus.append(preprocess_sentence(sentence))
print(len(corpus)) 

corpus[:10]

187088


['<start> eighteen years eighteen years <end>',
 '<start> she got one of your kids got you for eighteen years <end>',
 '<start> i know somebody paying child support for one of his kids <end>',
 '<start> his baby mama car and crib is bigger than his <end>',
 '<start> you will see him on tv any given sunday <end>',
 '<start> win the super bowl and drive off in a hyundai <end>',
 '<start> she was supposed to buy your shorty tyco with your money <end>',
 '<start> she went to the doctor got lipo with your money <end>',
 '<start> she walking around looking like michael with your money <end>',
 '<start> shouldve got that insured geico for your money <end>']

#### 토큰화

In [52]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 단어장 크기  
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    
    corpus = [x for x in corpus if len(x.split()) <= 15]

    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2424  433 ...    0    0    0]
 [   2   44   37 ...    3    0    0]
 [   2    4   34 ...    3    0    0]
 ...
 [   2  233    1 ...    0    0    0]
 [   2   10  500 ...    0    0    0]
 [   2  121   19 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f671c9ad890>


In [53]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [54]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2 2424  433 2424  433    3    0    0    0    0    0    0    0    0]
[2424  433 2424  433    3    0    0    0    0    0    0    0    0    0]


In [55]:
#tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성할 것입니다.

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### 3. 평가 데이터셋 분리

In [56]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, shuffle = True, random_state = 30)

In [57]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (133863, 14)
Target Train: (133863, 14)


### 4.인공지능 만들기

In [58]:
#모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [59]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)


<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 3.05832400e-05,  6.58686040e-05, -1.22613992e-05, ...,
         -6.23611686e-06,  9.34785130e-05, -4.33518981e-05],
        [ 3.43226100e-04, -3.31389965e-05, -2.83583031e-05, ...,
          4.25766993e-05,  2.06908633e-04, -6.13380907e-05],
        [ 8.68272327e-04, -1.18595155e-04,  1.55964313e-04, ...,
          8.49534190e-05,  1.92536434e-04, -3.43159714e-04],
        ...,
        [-8.67093098e-04, -4.66616504e-04, -5.73375844e-04, ...,
          7.11285102e-04, -2.18296261e-03,  6.92112444e-05],
        [-1.27605803e-03, -6.86682528e-04, -8.29066790e-04, ...,
          1.00520754e-03, -2.68615247e-03,  1.61742733e-04],
        [-1.58462080e-03, -9.13657481e-04, -1.07705477e-03, ...,
          1.31682388e-03, -3.13556660e-03,  1.97160203e-04]],

       [[ 3.05832400e-05,  6.58686040e-05, -1.22613992e-05, ...,
         -6.23611686e-06,  9.34785130e-05, -4.33518981e-05],
        [ 1.14522329e-04,  2.25281430e-04, -3

In [60]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [62]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
hist = model.fit(dataset, epochs=10,validation_data=(enc_val, dec_val))
print(hist.history['val_loss'])

Epoch 1/10
653/653 [==============================] - 92s 140ms/step - loss: 2.5487 - val_loss: 2.4235
Epoch 2/10
653/653 [==============================] - 91s 140ms/step - loss: 2.3913 - val_loss: 2.2807
Epoch 3/10
653/653 [==============================] - 93s 143ms/step - loss: 2.2625 - val_loss: 2.1548
Epoch 4/10
653/653 [==============================] - 91s 140ms/step - loss: 2.1448 - val_loss: 2.0375
Epoch 5/10
653/653 [==============================] - 92s 140ms/step - loss: 2.0388 - val_loss: 1.9387
Epoch 6/10
653/653 [==============================] - 91s 140ms/step - loss: 1.9443 - val_loss: 1.8454
Epoch 7/10
653/653 [==============================] - 91s 140ms/step - loss: 1.8582 - val_loss: 1.7622
Epoch 8/10
653/653 [==============================] - 91s 140ms/step - loss: 1.7785 - val_loss: 1.6871
Epoch 9/10
653/653 [==============================] - 91s 140ms/step - loss: 1.7045 - val_loss: 1.6141
Epoch 10/10
653/653 [==============================] - 91s 139ms/step - l

###  5.평가하기

In [63]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [67]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , i love you <end> '

### 총평

* val_loss가 2.2 이하여야 한다고 했는데, 3번째 EPOCH 에서 2.2 이하  수준으로 확인되었다. 

* 텍스트 마이닝 동영상 강의 자료가 빅데이터 R 분야가 많이 있어서, (대부분 영어 강의이거나) 강의가 없는 것이 아쉬웠다. 

* 지금까지 한 exploration 중에서 주석이 가장 꼼꼼하게 달린 친절한 노드였다. 강의가 없는 자기주도 학습이라면 이 정도 주석은 있어야, 구글링을 해도 시간 낭비가 덜하다. 

* LSTM모델을 두 번쨰 실습해 봤으니, 연휴 기간동안 비교해보고 꼼꼼히 다시 공부해 봐야 겠다. 